In [1]:
import pandas as pd
import numpy as np
import scipy
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.stats.multitest import multipletests
import statsmodels.stats.multicomp as multicomp

In [2]:
data_df = pd.read_excel('../data/42255_2025_1311_MOESM5_ESM.xlsx', sheet_name='2K')

In [5]:
data_df

,HFD,HFD+SAL,HFD+SANA
0,3530.0,4570.0,3595.0
1,4637.0,1647.0,4630.0
2,4378.0,1735.0,5460.0
3,2510.0,3845.0,2720.0
4,4555.0,3555.0,3471.0
5,2244.0,3659.0,3920.0
6,5365.0,2030.0,5000.0
7,2445.0,4935.0,4265.0
8,4600.0,4725.0,3247.0
9,3960.0,4050.0,8350.0


In [6]:
grp1, grp2, grp3 = data_df['HFD'].dropna().values, \
data_df['HFD+SAL '].dropna().values, \
data_df['HFD+SANA '].dropna().values

In [8]:
collect = [grp1, grp2, grp3]

In [10]:
def reorder_label(label):
    if label[-2:] == 'ND':
        return 'ND vs ' + label.split('vs')[0].strip()
    else:
        return label

In [11]:
for cat in [0]:
    
    print('N samples: ' + str(len(np.concatenate(collect))))

    print('ANOVA:')
    f = scipy.stats.f_oneway(grp1, grp2, grp3)
    print('F(2, ' + str(len(np.concatenate(collect))-3) + ') = ' + str(f.statistic) + ', p = ' + str(f.pvalue))

    #print('Bonferroni:')
    comp = multicomp.MultiComparison(data=np.concatenate(collect), 
                                      groups=np.concatenate([['HFD']*len(grp1), 
                                                             ['HFD+SAL']*len(grp2), 
                                                             ['HFD+SANA']*len(grp3)]))
    tbl, a1, a2 = comp.allpairtest(scipy.stats.ttest_ind, method= "bonf", alpha=0.05)
    bonf = pd.DataFrame(tbl)
    bonf.columns = bonf.loc[0].astype(str)
    bonf = bonf[1:]
    bonf['pval_corr'] = a1[2]
    bonf['g1'] = bonf['group1']
    bonf['g2'] = bonf['group2']
    bonf_df = bonf.copy()
    bonf_df['Bonferroni p-value'] = bonf_df['pval_corr']
    bonf_df['Comparison'] = bonf_df['g1'].astype(str) + ' vs ' + bonf_df['g2'].astype(str)
    bonf_df['Comparison'] = bonf_df['Comparison'].apply(reorder_label)

    #print('Tukey:')
    tukey = scipy.stats.tukey_hsd(*collect).pvalue
    tukey_df = pd.DataFrame({'g1':['HFD', 'HFD', 'HFD+SAL'], 
     'g2':['HFD+SAL', 'HFD+SANA', 'HFD+SANA'],
     'Tukey p-value':[tukey[0][1], tukey[0][2], tukey[1][2]]})
    tukey_df['Comparison'] = tukey_df['g1'].astype(str) + ' vs ' + tukey_df['g2'].astype(str)

    combo_df = pd.merge(bonf_df, tukey_df, on=['Comparison'], how='outer')
    combo_df['Published p-value'] = ''
    combo_df = combo_df[['Comparison', 'Published p-value', 'Bonferroni p-value', 'Tukey p-value']].set_index('Comparison')
    print(combo_df.to_markdown())
    
    for grp in collect:
        print(np.mean(grp))

N samples: 49
ANOVA:
F(2, 46) = 3.2535932948531525, p = 0.047686879741454115
| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |                     |            1         |       0.781656  |
| HFD vs HFD+SANA     |                     |            0.0742168 |       0.0455546 |
| HFD+SAL vs HFD+SANA |                     |            0.225213  |       0.158328  |
3353.0
3661.722222222222
4538.666666666667


F(2, 46) = 3.25, p = 0.0477
| Comparison          | Published p-value   |   Bonferroni p-value |   Tukey p-value |
|:--------------------|:--------------------|---------------------:|----------------:|
| HFD vs HFD+SAL      |          0.78           |            1.00         |       0.782  |
| HFD vs HFD+SANA     |              0.045       |            0.0742 |       0.0456 |
| HFD+SAL vs HFD+SANA |            0.16         |            0.225  |       0.158  |